# Clean csv files
cleaning up the data to only contain the necessary information

In [1]:
import os
import glob
import pandas as pd
import numpy as np
from datetime import datetime

In [6]:
def substract(a, b):                              
    return "".join(a.rsplit(b))

In [2]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [3]:
# number of fixed projects deleted
fixed_projects = 0

In [9]:
for filename in all_filenames:
    
    # read in a single csv
    df = pd.read_csv(filename)
    
    # limit to only hourly projects
    df.rate = df.rate.fillna(-1)
    fixed = df[ df['rate'] == -1].index
    df.drop(fixed, inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    # count how many projects are deleted
    fixed_projects += fixed.size
    
    # Transform date (delete the last three zeros)
    df.date = (df.date/1000).astype(int)
    df['date_2'] = df.date.apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d'))
    
    # Extract buyer information

    # Transform type str to dict
    import ast 
    res = ast.literal_eval(df.buyer[0])
    df['buyer'] = df.buyer.apply(lambda x: ast.literal_eval(x))

    # Country of employer
    country = []

    for i in df.buyer.index:
        try:
            country.append(df.buyer[i]['op_country'])
        except:
            country.append(np.nan)

    df['employer_country'] = country

    # city of employer
    city = []

    for i in df.buyer.index:
        try:
            city.append(df.buyer[i]['op_city'])
        except:
            city.append(np.nan)

    df['employer_city'] = city
    
    # drop assignment and candidate information
    df.drop(columns=['candidates','assignments','assignment_info','buyer','amount'], inplace=True)
    
    # Write clean csv file
    name = substract(filename,'.csv')
    df.to_csv("clean/" + name + "_clean"+".csv", index= False)
